<a href="https://colab.research.google.com/github/Yumlembam/Automatic-Essay-Scoring/blob/master/EssayScorinV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!ln -s "/gdrive/My Drive/Deep learning/Essay_Scoring_V2" "/content"

In [0]:
# upload the credential of kaggle account
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yumlembam","key":"dedaadf824834791cc388f3fe11a7370"}'}

In [0]:
#before importing the dataset we want to use this code
# The Kaggle API client expects this file to be in ~/.kaggle,
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c asap-aes

 58% 9.00M/15.6M [00:01<00:00, 11.5MB/s]
100% 15.6M/15.6M [00:01<00:00, 14.0MB/s]
 58% 11.0M/19.0M [00:00<00:00, 42.8MB/s]
100% 19.0M/19.0M [00:00<00:00, 63.3MB/s]
 78% 5.00M/6.39M [00:00<00:00, 29.6MB/s]
100% 6.39M/6.39M [00:00<00:00, 30.7MB/s]
  0% 0.00/5.05M [00:00<?, ?B/s]
100% 5.05M/5.05M [00:00<00:00, 46.3MB/s]
 82% 5.00M/6.12M [00:00<00:00, 34.0MB/s]
100% 6.12M/6.12M [00:00<00:00, 39.0MB/s]
  0% 0.00/2.06M [00:00<?, ?B/s]
100% 2.06M/2.06M [00:00<00:00, 140MB/s]
  0% 0.00/14.9k [00:00<?, ?B/s]
100% 14.9k/14.9k [00:00<00:00, 18.0MB/s]
  0% 0.00/14.9k [00:00<?, ?B/s]
100% 14.9k/14.9k [00:00<00:00, 15.2MB/s]
  0% 0.00/41.4k [00:00<?, ?B/s]
100% 41.4k/41.4k [00:00<00:00, 42.9MB/s]
  0% 0.00/88.5k [00:00<?, ?B/s]
100% 88.5k/88.5k [00:00<00:00, 94.2MB/s]
  0% 0.00/214k [00:00<?, ?B/s]
100% 214k/214k [00:00<00:00, 177MB/s]
 87% 48.0M/55.0M [00:00<00:00, 30.7MB/s]
100% 55.0M/55.0M [00:00<00:00, 59.5MB/s]
100% 5.00M/5.02M [00:00<00:00, 39.5MB/s]
100% 5.02M/5.02M [00:00<00:00, 31.9MB/s]


In [0]:
import os
import pandas as pd
from pandas import DataFrame

In [0]:
X = pd.read_csv('training_set_rel3.tsv', sep='\t', encoding='ISO-8859-1')
y = X['domain1_score']
X = X.dropna(axis=1)
X = X.drop(columns=['rater1_domain1', 'rater2_domain1'])

In [0]:
X.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [0]:
#Pre-Processing the model
import numpy as np
import nltk
import re
from nltk.corpus import stopwords

In [0]:
nltk.download('punkt') # Sentence tokenizer
nltk.download('stopwords') # English stop words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v) #re.sub(pattern,repl,string) #replace all the sting not in tha range a-z with empy.
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

In [0]:
def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle') #sentence tokenizer
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [0]:
print(X['essay'][0])
print("\n")
wordlist = essay_to_wordlist(X['essay'][0],True)
print(wordlist)
print(len(wordlist))
print("\n")
Sentences = essay_to_sentences(X['essay'][0],True)
print(Sentences)

Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the economy

In [0]:
!pip install emoji

In [0]:
!unzip "/gdrive/My Drive/Deep learning/Essay_Scoring_V2/data_files/glove.6B.50d.txt.zip"

Archive:  /gdrive/My Drive/Deep learning/Essay_Scoring_V2/data_files/glove.6B.50d.txt.zip
  inflating: glove.6B.50d.txt        
   creating: __MACOSX/
  inflating: __MACOSX/._glove.6B.50d.txt  


In [0]:
from Essay_Scoring_V2.emo_utils import *
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('/content/glove.6B.50d.txt')

For every word in an essay we will acess its feature vector in makeFeatureVec() using glove vector for word representation.We will add all the feature vector and take average.Since we are using glove of 50 dimension, each essay will be represented with a (1,50) dimensional vector.

In [0]:
def makeFeatureVec(words,  num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32") # to store the feature vector of each word
    num_words = 0
    for word in words:
        if word in word_to_index:
            num_words += 1
            featureVec = np.add(featureVec,word_to_vec_map[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays,num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay,num_features)
        counter = counter + 1
    return essayFeatureVecs

In [0]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten,Bidirectional
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(Bidirectional(LSTM(50, return_sequences=True,recurrent_dropout=0.4), input_shape=(1,50), merge_mode='concat'))
    #model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

In [0]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score

cv = KFold(n_splits=5, shuffle=True)
results = []
y_pred_list = []
cv_data =cv.split(X)
count = 1
print(cv_data)
for traincv, testcv in cv_data:
  
  print("\n--------Fold {}--------\n".format(count))
  X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
  train_essays = X_train['essay']
  test_essays = X_test['essay']
  num_features = 50 
  min_word_count = 40
  num_workers = 4
  context = 10
  downsampling = 1e-3
  clean_train_essays = []
  
  for essay_v in train_essays:
    clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords= True))
  trainDataVecs = getAvgFeatureVecs(clean_train_essays,num_features)
    
  clean_test_essays = []
    
  for essay_v in test_essays:
    clean_test_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True ))
  testDataVecs = getAvgFeatureVecs(clean_test_essays,num_features)
  
  trainDataVecs = np.array(trainDataVecs)
  
  testDataVecs = np.array(testDataVecs)
  
  # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
  
  trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
  
  testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
  
  lstm_model = get_model()
  lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=150)
    #lstm_model.load_weights('./model_weights/final_lstm.h5')
  y_pred = lstm_model.predict(testDataVecs)
    
    
  y_pred = np.around(y_pred)
    
  result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
  print("Kappa Score: {}".format(result))
  results.append(result)
  count = count + 1
  
  
  

<generator object _BaseKFold.split at 0x7fc1bfdfbf68>

--------Fold 1--------

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_12 (Bidirectio (None, 1, 100)            40400     
_________________________________________________________________
lstm_25 (LSTM)               (None, 64)                42240     
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 65        
Total params: 82,705
Trainable params: 82,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/150
10380/10380 [==============================] - 9s 826us/step - loss: 75.8590 - mean_absolute_error: 5.1505
Epoch 2/150
10380/10380 [==============================] - 2s 151us/step